In [1]:
# 约瑟夫环问题
# 假设有n个人， 从第一个人开始报数， 数到m的人被淘汰

# person = ['Bob', 'Jack', 'Allen', 'Tony', 'Peter', 'Rose', 'Wade']


class Josephus:

    class Person:
        def __init__(self, name, id, age):
            self.name = name
            self.id = id
            self.age = age
        def print_info(self):
            print("id:{}, name:{}, age:{}" .format(self.id, self.name, self.age))

    def __init__(self, participants, step, start):
        self.participants = participants
        self.step = step
        self.start = start

    def josephus(self):
        n = len(self.participants)
        # 将所有人的序号按顺序（1 ~ n）生成一个列表
        sequence = list(range(1, n+1))
        index = self.start - 1
        result = []
        for i in range(n):
            count = 0
            while count < self.step:
                if sequence[index] != 0:
                    count += 1
                if count == self.step:
                    result.append(self.participants[index])
                    sequence[index] = 0
                index = (index + 1) % n
        return result   

    def print_result(self, reslut):
        for i in range(len(reslut) - 1):
            print("第%d位被淘汰" %(i + 1))   
            reslut[i].print_info()
        print("获胜者：")
        reslut[len(reslut) - 1].print_info()

p1 = Josephus.Person('Bob', 1, 15)
p2 = Josephus.Person('Jack', 2, 12)
p3 = Josephus.Person('Allen', 3, 17)
p4 = Josephus.Person('Tony', 4, 20)
p5 = Josephus.Person('Peter', 5, 15)
p6 = Josephus.Person('Rose', 6, 16)
p7 = Josephus.Person('Wade', 7, 19)
participants = [p1, p2, p3, p4, p5, p6, p7]

jos = Josephus(participants, 4, 3)
rank = jos.josephus()
jos.print_result(rank)

第1位被淘汰
id:6, name:Rose, age:16
第2位被淘汰
id:3, name:Allen, age:17
第3位被淘汰
id:1, name:Bob, age:15
第4位被淘汰
id:7, name:Wade, age:19
第5位被淘汰
id:2, name:Jack, age:12
第6位被淘汰
id:5, name:Peter, age:15
获胜者：
id:4, name:Tony, age:20
